In [1]:
import os 
from pathlib import Path
import logging

In [2]:
%pwd

'f:\\ML_Ops_learning\\Mice-Protein-Expression\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\ML_Ops_learning\\Mice-Protein-Expression'

In [ ]:
project_name="mice"

In [5]:
project_name="mice"

list_of_files = [
    ".github/workflows/main.yaml",
    f"src/{project_name}/__init__.py",
    
    f"src/{project_name}/components/__init__.py",
    f"src/{project_name}/components/data_ingestion.py",
    f"src/{project_name}/components/data_validation.py",
    f"src/{project_name}/components/data_transformation.py",
    f"src/{project_name}/components/model_trainer.py",
    f"src/{project_name}/components/model_evaluation.py",
    
    f"src/{project_name}/config/__init__.py",
    
    f"src/{project_name}/constants/__init__.py",

    f"src/{project_name}/entity/__init__.py",

    f"src/{project_name}/pipeline/__init__.py",
    f"src/{project_name}/pipeline/stage_01_data_ingestion.py",
    f"src/{project_name}/pipeline/stage_02_data_validation.py",
    f"src/{project_name}/pipeline/stage_03_data_transformation.py",
    f"src/{project_name}/pipeline/stage_04_model_trainer.py",
    f"src/{project_name}/pipeline/stage_05_model_evaluation.py",
    
    f"src/{project_name}/utils/__init__.py",    
    "config/config.yaml",
    "params.yaml",
    "dvc.yaml",

    "app.py",
    "main.py",
    "requirements.txt",
    "setup.py",
    "templates/index.html"
]

In [6]:
for filepath in list_of_files:
    filepath = Path(filepath)
    filedir, filename = os.path.split(filepath)

    if filedir != "":
        os.makedirs(filedir, exist_ok=True)
        logging.info(f"Creating directory:{filedir} for the file {filename}")
    
    if (not os.path.exists(filepath)) or (os.path.getsize(filepath) == 0):
        with open(filepath,'w') as f:
            pass
            logging.info(f"Creating empty file: {filepath}")
    else:
        logging.info(f"{filename} is already exists")

## requirement.txt

In [7]:
%%writefile -a requirements.txt
tensorflow
pandas 
dvc
mlflow
notebook
numpy
matplotlib
seaborn
python-box
pyYAML
tqdm
ensure
joblib
types-PyYAML
scipy
Flask
Flask-Cors
gdown
-e .


Appending to requirements.txt


# setup.py

In [9]:
%%writefile -a setup.py
import setuptools

with open("README.md", "r", encoding="utf-8") as f:
    long_description = f.read()


__version__ = "0.0.0"

GITHUB_REPO_NAME = " Mice-Protein-Expression"
GITHUB_AUTHOR_USER_NAME = "SunilKumar-ugra"
SRC_REPO = "mice"
GITHUB_AUTHOR_EMAIL = "ugargolsunilkumar@gmail.com"



setuptools.setup(
    name=SRC_REPO,
    version=__version__,
    author=GITHUB_AUTHOR_USER_NAME,
    author_email=GITHUB_AUTHOR_EMAIL,
    description="Mice Protein Expression",
    long_description=long_description,
    long_description_content="text/markdown",
    url=f"https://github.com/{GITHUB_AUTHOR_USER_NAME}/{GITHUB_REPO_NAME}",
    project_urls={
        "Bug Tracker": f"https://github.com/{GITHUB_AUTHOR_USER_NAME}/{GITHUB_REPO_NAME}/issues",
    },
    package_dir={"": "src"},
    packages=setuptools.find_packages(where="src")
)

Appending to setup.py


In [10]:
%pip install -r requirements.txt

Obtaining file:///F:/ML_Ops_learning/Mice-Protein-Expression (from -r requirements.txt (line 10))
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py develop for mice
Note: you may need to restart the kernel to use updated packages.


## Custom Logging

In [11]:
%%writefile -a src/{project_name}/__init__.py
import logging
import os
from datetime import datetime
import sys

LOG_FILE=f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"
logs_path=os.path.join(os.getcwd(),"logs",LOG_FILE)
os.makedirs(logs_path,exist_ok=True)

LOG_FILE_PATH=os.path.join(logs_path,LOG_FILE)

logging.basicConfig(
    format="[%(asctime)s] - %(name)s - %(levelname)s - %(module)s file - LineNum:%(lineno)d   - %(message)s",
    level=logging.INFO,
    handlers=[
        logging.FileHandler(LOG_FILE_PATH),
        logging.StreamHandler(sys.stdout)
    ]
)

logger=logging.getLogger("mice")

Appending to src/mice/__init__.py


## Utility

In [12]:
%%writefile -a src/{project_name}/utils/__init__.py
import os
from box.exceptions import BoxValueError
import yaml
from mice import logger
import json
import joblib
from ensure import ensure_annotations
from box import ConfigBox
from pathlib import Path
from typing import Any
import base64



@ensure_annotations
def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """reads yaml file and returns

    Args:
        path_to_yaml (str): path like input

    Raises:
        ValueError: if yaml file is empty
        e: empty file

    Returns:
        ConfigBox: ConfigBox type
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)
            logger.info(f"yaml file: {path_to_yaml} loaded successfully")
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e
    


@ensure_annotations
def create_directories(path_to_directories: list, verbose=True):
    """create list of directories

    Args:
        path_to_directories (list): list of path of directories
        ignore_log (bool, optional): ignore if multiple dirs is to be created. Defaults to False.
    """
    for path in path_to_directories:
        os.makedirs(path, exist_ok=True)
        if verbose:
            logger.info(f"created directory at: {path}")


@ensure_annotations
def save_json(path: Path, data: dict):
    """save json data

    Args:
        path (Path): path to json file
        data (dict): data to be saved in json file
    """
    with open(path, "w") as f:
        json.dump(data, f, indent=4)

    logger.info(f"json file saved at: {path}")




@ensure_annotations
def load_json(path: Path) -> ConfigBox:
    """load json files data

    Args:
        path (Path): path to json file

    Returns:
        ConfigBox: data as class attributes instead of dict
    """
    with open(path) as f:
        content = json.load(f)

    logger.info(f"json file loaded succesfully from: {path}")
    return ConfigBox(content)


@ensure_annotations
def save_bin(data: Any, path: Path):
    """save binary file

    Args:
        data (Any): data to be saved as binary
        path (Path): path to binary file
    """
    joblib.dump(value=data, filename=path)
    logger.info(f"binary file saved at: {path}")


@ensure_annotations
def load_bin(path: Path) -> Any:
    """load binary data

    Args:
        path (Path): path to binary file

    Returns:
        Any: object stored in the file
    """
    data = joblib.load(path)
    logger.info(f"binary file loaded from: {path}")
    return data

@ensure_annotations
def get_size(path: Path) -> str:
    """get size in KB

    Args:
        path (Path): path of the file

    Returns:
        str: size in KB
    """
    size_in_kb = round(os.path.getsize(path)/1024)
    return f"~ {size_in_kb} KB"

Appending to src/mice/utils/__init__.py


## Params YAML

In [13]:
%%writefile params.yaml
ElasticNet:
  alpha: 0.2
  l1_ratio: 0.1

Overwriting params.yaml


Schema yaml

## Intializing Path in Constant Constructor File

In [15]:
%%writefile  src/{project_name}/constants/__init__.py
from pathlib import Path
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")

Overwriting src/mice/constants/__init__.py
